In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
from scipy.stats import pearsonr, spearmanr
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.preprocessing import MinMaxScaler

def analyze_technical_correlations(ticker):
    # Step 1: Download stock data
    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=60)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Step 2: Add all TA-lib indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df,
                             open="Open", high="High", low="Low",
                             close="Close", volume="Volume",
                             fillna=True)

    df.reset_index(inplace=True)
    
    # Step 3: Normalize data
    print("📊 Normalizing numeric features...")
    df['Returns'] = df['Close'].pct_change()
    df['Lagged_Returns'] = df['Returns'].shift(1)
    df.dropna(inplace=True)

    scaler = MinMaxScaler()
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Step 4: Select indicator columns (exclude target variables)
    exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Returns', 'Lagged_Returns']
    indicator_cols = [col for col in df.columns if col not in exclude_cols]

    # Step 5: Correlation Analysis
    print("🔍 Calculating correlations...")
    pearson_results = {'Price': {}, 'Returns': {}, 'Lagged_Returns': {}}
    spearman_results = {'Price': {}, 'Returns': {}, 'Lagged_Returns': {}}

    def calculate_correlations(data, x_col, y_col, lag=0):
        x = data[x_col].shift(lag) if lag > 0 else data[x_col]
        y = data[y_col]
        valid_data = pd.concat([x, y], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
        if len(valid_data) < 3:
            return np.nan, np.nan
        try:
            p_corr, _ = pearsonr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
            s_corr, _ = spearmanr(valid_data.iloc[:, 0], valid_data.iloc[:, 1])
            return p_corr, s_corr
        except:
            return np.nan, np.nan

    for col in indicator_cols:
        for label, y_col, lag in [('Price', 'Close', 0), ('Returns', 'Returns', 0), ('Lagged_Returns', 'Returns', 1)]:
            p_corr, s_corr = calculate_correlations(df, col, y_col, lag)
            pearson_results[label][col] = p_corr
            spearman_results[label][col] = s_corr

    pearson_df = pd.DataFrame(pearson_results)
    spearman_df = pd.DataFrame(spearman_results)

    # Step 6: Save correlation data
    # pearson_df.to_csv(f"{ticker}_pearson_correlations.csv")
    # spearman_df.to_csv(f"{ticker}_spearman_correlations.csv")

    # # Step 7: Visualize with heatmaps
    # print("📈 Creating heatmaps...")
    # plt.figure(figsize=(14, 10))
    # sns.heatmap(pearson_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    # plt.title(f"{ticker} - Pearson Correlation Heatmap")
    # plt.tight_layout()
    # plt.savefig(f"{ticker}_pearson_heatmap.png")
    # plt.show()

    # plt.figure(figsize=(14, 10))
    # sns.heatmap(spearman_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    # plt.title(f"{ticker} - Spearman Correlation Heatmap")
    # plt.tight_layout()
    # plt.savefig(f"{ticker}_spearman_heatmap.png")
    # plt.show()

    # Step 8: Show strong correlations
    print("\n📌 Significant Pearson Correlations (|corr| > 0.7):")
    print(pearson_df[(pearson_df['Price'].abs() > 0.7)])
    # store them in a file 
    pearson_df[(pearson_df['Price'].abs() > 0.7)].to_csv(f"Pearsons_Coeff_Sixty_Day/{ticker}_significant_pearson.csv")

    print("\n📌 Significant Spearman Correlations (|corr| > 0.7):")
    print(spearman_df[(spearman_df['Returns'].abs() > 0.7)])



    # def plot_significant_indicators(df, indicator_list):
    #     print("\n📉 Plotting significant indicators with Close price...")
    #     df_plot = df.set_index('Date')
    #     for indicator in indicator_list:
    #         if indicator not in df_plot.columns:
    #             continue
    #         plt.figure(figsize=(14, 6))
    #         plt.plot(df_plot['Close'], label='Close Price', linewidth=2)
    #         plt.plot(df_plot[indicator], label=indicator, linestyle='--')
    #         plt.title(f"{ticker} - Close vs {indicator}")
    #         plt.xlabel("Date")
    #         plt.ylabel("Normalized Value")
    #         plt.legend()
    #         plt.tight_layout()
    #         # plt.savefig(f"{ticker}_Close_vs_{indicator}.png")
    #         plt.show()

    # significant_indicators = pearson_df.index[pearson_df['Price'].abs() > 0.7].tolist()
    # plot_significant_indicators(df, significant_indicators)


In [9]:
analyze_technical_correlations("TCS.NS")  #Large Cap IT
analyze_technical_correlations("INFY.NS") #large Cap IT
analyze_technical_correlations("WIPRO.NS")  #Large Cap IT
analyze_technical_correlations("HCLTECH.NS") # Large Cap IT

analyze_technical_correlations("TECHM.NS")  # Mid Cap IT
analyze_technical_correlations("PERSISTENT.NS")  # Mid Cap IT
# Small Cap IT
analyze_technical_correlations("LTIM.NS")  # Small Cap IT


📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInput

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.7):
                         Price   Returns  Lagged_Returns
volume_vpt            0.727524  0.351008       -0.296043
trend_vortex_ind_pos  0.824598 -0.093092       -0.290161
others_cr             1.000000  0.314618       -0.445818

📌 Significant Spearman Correlations (|corr| > 0.7):
               Price  Returns  Lagged_Returns
others_dr   0.182753      1.0       -0.102627
others_dlr  0.182753      1.0       -0.102627
📥 Downloading data for INFY.NS...


[*********************100%***********************]  1 of 1 completed
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInput

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.7):
                         Price   Returns  Lagged_Returns
volume_obv            0.873597  0.022340       -0.138267
volume_fi             0.817850  0.249060       -0.302609
volume_vpt            0.982620  0.128443       -0.315356
volume_vwap           0.873464 -0.164643       -0.110494
volume_nvi            0.780046  0.028678       -0.028381
volatility_bbm        0.863753 -0.127595       -0.062785
volatility_bbh        0.881831 -0.164130       -0.178855
volatility_kcc        0.884470 -0.163035       -0.089080
volatility_kch        0.880529 -0.162379       -0.112926
volatility_kcl        0.881575 -0.162438       -0.066796
volatility_dcl        0.753947 -0.055285        0.000429
volatility_dch        0.921437 -0.077471       -0.221149
volatility_dcm        0.914161 -0.072684       -0.127465
trend_macd            0.769104 -0.167844      

...

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.7):
                         Price   Returns  Lagged_Returns
volume_adi            0.770148 -0.098951       -0.215752
volume_obv            0.857373 -0.041337       -0.132039
volume_vpt            0.983075 -0.116507       -0.204604
volume_vwap           0.833724 -0.266587       -0.179067
volume_nvi            0.811644 -0.011093       -0.127500
volatility_bbm        0.823781 -0.214829       -0.097793
volatility_bbh        0.887800 -0.319925       -0.245374
volatility_kcc        0.858535 -0.280489       -0.172750
volatility_kch        0.866527 -0.287640       -0.190386
volatility_kcl        0.845214 -0.272334       -0.156973
volatility_dch        0.943220 -0.210390       -0.194886
volatility_dcm        0.845621 -0.157483       -0.103265
trend_sma_fast        0.854543 -0.270308       -0.168951
trend_sma_slow        0.866803 -0.194952      

[*********************100%***********************]  1 of 1 completed
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInput

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.7):
                         Price   Returns  Lagged_Returns
volume_vpt            0.966627 -0.045444       -0.327457
volume_vwap           0.778937 -0.254448       -0.193011
volatility_bbm        0.800532 -0.212798       -0.137270
volatility_bbh        0.871044 -0.273544       -0.244172
volatility_kcc        0.837998 -0.273763       -0.196337
volatility_kch        0.838931 -0.285378       -0.218150
volatility_kcl        0.830151 -0.261715       -0.176507
volatility_dch        0.876723 -0.205140       -0.239940
volatility_dcm        0.798001 -0.163353       -0.144760
trend_sma_fast        0.828324 -0.261227       -0.190231
trend_sma_slow        0.848064 -0.198140       -0.126008
trend_ema_fast        0.881450 -0.242798       -0.197663
trend_ema_slow        0.842865 -0.215342       -0.155734
trend_vortex_ind_pos  0.883377 -0.111571      

[*********************100%***********************]  1 of 1 completed
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/Users/devishadhanuka/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInput

🧠 Computing all technical indicators...
📊 Normalizing numeric features...
🔍 Calculating correlations...

📌 Significant Pearson Correlations (|corr| > 0.7):
                         Price   Returns  Lagged_Returns
volume_obv            0.745371  0.177127       -0.194365
volume_cmf           -0.761371  0.256937       -0.094431
volume_vpt            0.974010 -0.033079       -0.156677
volume_vwap           0.958771 -0.192835       -0.116779
volume_nvi            0.961754 -0.091010       -0.096316
volatility_bbm        0.946790 -0.178633       -0.094384
volatility_bbh        0.962560 -0.209220       -0.158944
volatility_bbl        0.811636 -0.121584       -0.007400
volatility_kcc        0.963271 -0.198398       -0.100750
volatility_kch        0.966552 -0.198045       -0.103138
volatility_kcl        0.959324 -0.198541       -0.098461
volatility_kcw       -0.815272  0.186367        0.056282
volatility_dcl        0.867002 -0.131323       -0.082779
volatility_dch        0.986608 -0.140563      

SENSITIVITY ANALYSIS

In [13]:
def sensitivity_analysis(ticker, period=1):
    import pandas as pd
    import numpy as np
    import yfinance as yf
    import matplotlib.pyplot as plt
    from ta import add_all_ta_features
    from ta.utils import dropna
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    from scipy.stats import zscore

    print(f"📥 Downloading data for {ticker}...")
    end_date = datetime.today()
    start_date = end_date - relativedelta(days=90)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if df.empty:
        print(f"❌ No data retrieved for {ticker}.")
        return

    df.reset_index(inplace=True)
    df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Step 2: Add technical indicators
    print("🧠 Computing all technical indicators...")
    df = dropna(df)
    df = add_all_ta_features(df,
                             open="Open", high="High", low="Low",
                             close="Close", volume="Volume",
                             fillna=True)
    df.reset_index(inplace=True)

    # Z-score normalization for numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(zscore)
    df.dropna(inplace=True)

    # Calculate price change
    df['Price_Change'] = df['Close'].diff(periods=period)
    df.dropna(inplace=True)

    # Identify indicator columns (exclude OHLCV and computed returns)
    exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Price_Change']
    indicator_cols = [col for col in df.columns if col not in exclude_cols]

    # Compute sensitivity scores
    sensitivity_scores = {}

    for col in indicator_cols:
        if df[col].isnull().any():
            continue
        df[f'{col}_Change'] = df[col].diff(periods=period)
        df_filtered = df[[f'{col}_Change', 'Price_Change']].replace([np.inf, -np.inf], np.nan).dropna()

        # Avoid divide-by-zero
        df_filtered = df_filtered[df_filtered[f'{col}_Change'] != 0]

        if df_filtered.empty:
            sensitivity_scores[col] = np.nan
            continue

        # Clip extreme values to prevent distortion
        sensitivities = (df_filtered['Price_Change'] / df_filtered[f'{col}_Change']).clip(lower=-10, upper=10)
        sensitivity_scores[col] = sensitivities.mean()

    # Construct sensitivity DataFrame
    sensitivity_df = pd.DataFrame.from_dict(sensitivity_scores, orient='index', columns=['Average_Sensitivity'])
    sensitivity_df.dropna(inplace=True)

    # Compute Z-score of sensitivities
    sensitivity_df['Zscore_Sensitivity'] = zscore(sensitivity_df['Average_Sensitivity'])

    # Sort for visualization
    sensitivity_df.sort_values(by='Average_Sensitivity', ascending=False, inplace=True)
    top_raw = sensitivity_df.head(20)

    sensitivity_df.sort_values(by='Zscore_Sensitivity', ascending=False, inplace=True)
    top_zscore = sensitivity_df.head(20)

    # --- Plot Raw + Z-score Side-by-Side ---
    # fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    # Plot 1: Raw Sensitivity Scores
    # top_raw['Average_Sensitivity'].plot(kind='bar', ax=axes[0])
    # axes[0].set_title(f"{ticker} - Top 20 Raw Sensitivity Scores")
    # axes[0].set_ylabel("Average ΔPrice / ΔIndicator")
    # axes[0].set_xlabel("Technical Indicator")
    # axes[0].tick_params(axis='x', rotation=90)
    # axes[0].grid(True)

    # Plot 2: Z-score Normalized Sensitivity
    # top_zscore['Zscore_Sensitivity'].plot(kind='bar', ax=axes[1], color='orange')
    # axes[1].set_title(f"{ticker} - Top 20 Normalized (Z-score) Sensitivity")
    # axes[1].set_ylabel("Z-score of Sensitivity")
    # axes[1].set_xlabel("Technical Indicator")
    # axes[1].tick_params(axis='x', rotation=90)
    # axes[1].grid(True)

    # plt.tight_layout()
    # plt.savefig(f"{ticker}_sensitivity_comparison.png")
    # plt.show()

    # 📋 Print top 20 indicators
    print("\n📌 Top 20 Indicators by Raw Average Sensitivity:")
    print(top_raw[['Average_Sensitivity']])

    print("\n📌 Top 20 Indicators by Z-score Normalized Sensitivity:")
    print(top_zscore[['Zscore_Sensitivity']])
    # store top 20 Z-score indicators in a file
    top_zscore[['Zscore_Sensitivity']].to_csv(f"Sensitivity_60/{ticker}_top_sensitivity_indicators.csv")

    # Optional: Save to CSV
    # sensitivity_df.to_csv(f"{ticker}_sensitivity_scores.csv")

    return sensitivity_df


In [14]:
sensitivity_analysis("TCS.NS", period=1)
sensitivity_analysis("INFY.NS", period=1)
sensitivity_analysis("WIPRO.NS", period=1)
sensitivity_analysis("HCLTECH.NS", period=1)
sensitivity_analysis("TECHM.NS", period=1)
sensitivity_analysis("PERSISTENT.NS", period=1)
sensitivity_analysis("LTIM.NS", period=1)

📥 Downloading data for TCS.NS...


[*********************100%***********************]  1 of 1 completed


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                       Average_Sensitivity
momentum_tsi                      2.294163
volume_vpt                        1.854567
trend_kst                         1.572366
momentum_rsi                      1.537790
trend_aroon_ind                   1.451901
momentum_ao                       1.446071
trend_ema_fast                    1.443943
momentum_kama                     1.409822
volume_nvi                        1.394672
volume_sma_em                     1.215497
momentum_stoch_rsi_k              1.203431
trend_adx_pos                     1.149208
trend_ichimoku_a                  1.144070
trend_vortex_ind_diff             1.099720
trend_cci                         1.066570
trend_kst_diff                    1.053283
trend_sma_slow                    1.029747
others_cr                         1.000000
volatility_dcp                    0.962440
momentum_stoch_rsi                0.937596

📌 Top 2

[*********************100%***********************]  1 of 1 completed


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                      Average_Sensitivity
trend_ichimoku_b                 1.984622
volatility_dcl                   1.893499
trend_aroon_ind                  1.745005
volume_vpt                       1.617993
trend_adx                        1.580439
momentum_kama                    1.511227
volatility_dcm                   1.325164
momentum_ppo_signal              1.240192
trend_macd_signal                1.225186
trend_macd                       1.212264
trend_sma_fast                   1.200443
momentum_rsi                     1.179562
trend_adx_pos                    1.177760
volatility_bbp                   1.145156
momentum_roc                     1.036442
volume_obv                       1.017217
momentum_ppo_hist                1.009957
trend_vortex_ind_pos             1.000724
others_cr                        1.000000
momentum_stoch_rsi               0.981854

📌 Top 20 Indicators by Z-sco

...

[*********************100%***********************]  1 of 1 completed


🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                         Average_Sensitivity
volume_vpt                          3.053193
volume_fi                           1.604214
volume_sma_em                       1.408940
trend_adx_pos                       1.260871
volatility_dcw                      1.229263
volatility_bbl                      1.081056
volatility_kcc                      1.042597
others_cr                           1.000000
momentum_rsi                        0.989321
momentum_stoch_rsi                  0.974491
trend_adx                           0.969010
momentum_stoch_signal               0.962124
volatility_dcm                      0.937549
volatility_dcl                      0.925343
momentum_ppo_hist                   0.924217
trend_macd_diff                     0.885198
trend_kst_diff                      0.846530
volatility_kcl                      0.807283
trend_visual_ichimoku_b             0.774198
trend_ema_f

[*********************100%***********************]  1 of 1 completed

🧠 Computing all technical indicators...

📌 Top 20 Indicators by Raw Average Sensitivity:
                     Average_Sensitivity
volume_vpt                      2.053619
volatility_dch                  1.865482
trend_ichimoku_b                1.744374
volatility_dcm                  1.552207
momentum_tsi                    1.335869
trend_ichimoku_conv             1.244373
trend_macd                      1.113297
momentum_kama                   1.067625
trend_ichimoku_base             1.063925
trend_ichimoku_a                1.058966
others_cr                       1.000000
volatility_kch                  0.857071
volatility_bbp                  0.851466
trend_psar_up                   0.850664
trend_cci                       0.813804
trend_aroon_ind                 0.803121
momentum_rsi                    0.800719
momentum_ao                     0.784000
trend_kst                       0.774330
volatility_kcc                  0.760839

📌 Top 20 Indicators by Z-score Normalized Sensiti

,Average_Sensitivity,Zscore_Sensitivity
volume_vpt,2.053619,2.444279
volatility_dch,1.865482,2.174751
trend_ichimoku_b,1.744374,2.001249
volatility_dcm,1.552207,1.725947
momentum_tsi,1.335869,1.416018
...,...,...
volatility_kcw,-0.800445,-1.644509
volatility_ui,-0.848704,-1.713646
volatility_bbh,-0.887285,-1.768917
trend_psar_down,-1.174796,-2.180811
